# Self-Learning 5.1 : Improving integral accuracy by "edge correction"

## Motivation  

Consider an integral 
$I(a,b) = \int_a^b dx f(x)$.  
We discretize the integration domain into N parts:  
$x_0=a, x_1=a+h, \dots, x_k=a+kh, \dots, x_N=b$ with $h=(b-a)/N$.  

The most naive formula is approximating the integral by sum of rectangles,  
$I_{\rm rec} \equiv h\sum_{k=1}^N f(x_k)$.  
Using trapezoids instead of rectangles generally improves the accuracy, and the trapezoid method turns out to simply add edge values to $I_{\rm rec}$,  
$I_{\rm tra} \equiv I_{\rm rec} - \frac{1}{2}h(f(b)-f(a))$.  
The analysis given in the textbook shows that the error of $I_{\rm tra}$ (i.e. its difference from the true value $I(a,b)$) is $\mathcal O(h^2)$,  
$I(a,b) = I_{\rm tra} + \mathcal O(h^2)$.  
Adding the Euler-Maclaurin term to $I_{\rm tra}$,  
$I_{\rm em} \equiv I_{\rm tra} + \frac{1}{12}h^2(f'(b)-f'(a))$,  
further reduces the error to $\mathcal O(h^4)$, that is,  
$I(a,b) = I_{\rm em} + \mathcal O(h^4)$. 

I felt the fact above somewhat mysterious;  integration is inherently a __bulk__ operation ― it measures the area under the curve $f(x)$ between a and b.  But the fact above says that we can reduce the error of this bulk quantity by adding __edge__ values, f(a), f(b), f'(a), f'(b) and so on. 

Is that true?  

## Task  

Numerically calculate each of $I_{\rm rec}, I_{\rm tra}, I_{\rm em}$ for various integrations, and check if  
 - the accuracy improves in the order of $I_{\rm rec}, I_{\rm tra}, I_{\rm em}$
 - the errors of $I_{\rm tra}$ and $I_{\rm em}$ are $\mathcal O(h^2)$ and $\mathcal O(h^4)$, respectively.
 - by decreasing the step size $h$ to, say $h/2$, the error reduces by 1/4 and 1/16 for $I_{\rm tra}$ and $I_{\rm em}$, respectively

In [1]:
import math

In [2]:
def calculate_integral(f, df, a, b, N, method):
    """
    Returns integral value of a function f (with its derivative function df) from a to b.  
    N represents the number of division of the integral domain [a, b]
    "method" is a string selecting a integration method from "rectangle", "trapezoid", "euler-maclaurin" 
    """
    assert a<b
    assert method in ["rectangle", "trapezoid", "euler-maclaurin"]
    
    h = (b-a)/N
    
    result = 0.
    for k in range(1,N+1):
        x_k = a + h*k
        result += h*f(x_k)
    if method == "rectangle":
        return result
    result += 0.5*h*(f(a) - f(b))  # trapezoid
    if method == "trapezoid":
        return result
    result += 1/12*h*h*(df(a) - df(b))  # euler-maclaurin
    return result

def print_result(f, df, a, b, true_value, N):
    """
    this is a wrapper of def calculate_integral
    true_value is the analytical result of the integral you should perform by hand beforehand.
    For each of method of "rectangle", "trapezoid", "euler-maclaurin", 
    this function prints 
       the numerically calculated integral value
       the error (i.e. the absolute difference from the true_value)
       the order of the error, i.e., log_h(the error)
    """
    print(f"true_value:{true_value}")
    h = (b-a)/N
    for method in ["rectangle", "trapezoid", "euler-maclaurin"]:
        integral = calculate_integral(f, df, a, b, N, method=method)
        error = abs(integral - true_value)
        error_order = math.log(error, h)

        integral_2 = calculate_integral(f, df, a, b, 2*N, method=method)
        error_2 = abs(integral_2 - true_value)
        alpha = math.log(error/error_2, 2)
        
        space = ""
        if method in ["rectangle", "trapezoid"]:
            space += " "*(len("euler-maclaurin") - len("rectangle"))  # just to align printing
        print(f"method:{method},{space} integral:{integral:.10f}, error:{error:.20f}, error_order:{error_order:.2f}, error_change_rate:{alpha:.4f}")

$\int_0^2 dx (x^4 - 2x + 1) = \frac{22}{5}$  

In [3]:
print_result(lambda x:x**4-2*x+1, lambda x:4*x**3-2, a=0, b=2, true_value=22/5, N=100)

true_value:4.4
method:rectangle,       integral:4.5210666560, error:0.12106665599999910654, error_order:0.54, error_change_rate:1.0064
method:trapezoid,       integral:4.4010666560, error:0.00106665599999899996, error_order:1.75, error_change_rate:2.0000
method:euler-maclaurin, integral:4.3999999893, error:0.00000001066666754923, error_order:4.69, error_change_rate:4.0000


In [4]:
print_result(lambda x:x**4-2*x+1, lambda x:4*x**3-2, a=0, b=2, true_value=22/5, N=1000)  # changed N to 1000

true_value:4.4
method:rectangle,       integral:4.4120106667, error:0.01201066666559835738, error_order:0.71, error_change_rate:1.0006
method:trapezoid,       integral:4.4000106667, error:0.00001066666559879081, error_order:1.84, error_change_rate:2.0000
method:euler-maclaurin, integral:4.4000000000, error:0.00000000000106759046, error_order:4.44, error_change_rate:4.0613


$\int_0^{\pi/2} dx \sin x = 1$  

In [5]:
print_result(lambda x:math.sin(x), lambda x:math.cos(x), a=0, b=math.pi/2, true_value=1., N=100)

true_value:1.0
method:rectangle,       integral:1.0078334199, error:0.00783341987358232572, error_order:1.17, error_change_rate:0.9981
method:trapezoid,       integral:0.9999794382, error:0.00002056176039211266, error_order:2.60, error_change_rate:2.0000
method:euler-maclaurin, integral:0.9999999999, error:0.00000000008455647293, error_order:5.58, error_change_rate:3.9999


In [6]:
print_result(lambda x:math.sin(x), lambda x:math.cos(x), a=0, b=math.pi/2, true_value=1., N=1000)  # changed N to 1000

true_value:1.0
method:rectangle,       integral:1.0007851925, error:0.00078519254663111937, error_order:1.11, error_change_rate:0.9998
method:trapezoid,       integral:0.9999997944, error:0.00000020561676628006, error_order:2.38, error_change_rate:2.0000
method:euler-maclaurin, integral:1.0000000000, error:0.00000000000000788258, error_order:5.03, error_change_rate:2.8278


$\int_0^{\pi} dx \sin x = 2$ 

In [7]:
print_result(lambda x:math.sin(x), lambda x:math.cos(x), a=0, b=math.pi, true_value=2., N=100)

true_value:2.0
method:rectangle,       integral:1.9998355039, error:0.00016449611255642260, error_order:2.52, error_change_rate:2.0000
method:trapezoid,       integral:1.9998355039, error:0.00016449611255642260, error_order:2.52, error_change_rate:2.0000
method:euler-maclaurin, integral:1.9999999973, error:0.00000000270587152684, error_order:5.70, error_change_rate:4.0000


In [8]:
print_result(lambda x:math.sin(x), lambda x:math.cos(x), a=0, b=math.pi, true_value=2., N=1000) # changed N to 1000

true_value:2.0
method:rectangle,       integral:1.9999983551, error:0.00000164493433629787, error_order:2.31, error_change_rate:2.0000
method:trapezoid,       integral:1.9999983551, error:0.00000164493433629787, error_order:2.31, error_change_rate:2.0000
method:euler-maclaurin, integral:2.0000000000, error:0.00000000000026934011, error_order:5.02, error_change_rate:4.0349


$\int_1^3 dx \frac{1}{x} = \log 3$

In [9]:
print_result(lambda x:1/x, lambda x:-1/(x*x), a=1, b=3, true_value=math.log(3), N=100)

true_value:1.0986122886681098
method:rectangle,       integral:1.0919752503, error:0.00663703835365647699, error_order:1.28, error_change_rate:0.9968
method:trapezoid,       integral:1.0986419170, error:0.00002962831301012159, error_order:2.67, error_change_rate:2.0000
method:euler-maclaurin, integral:1.0986122874, error:0.00000000131661948011, error_order:5.23, error_change_rate:3.9998


In [10]:
print_result(lambda x:1/x, lambda x:-1/(x*x), a=1, b=3, true_value=math.log(3), N=1000) # changed N to 1000

true_value:1.0986122886681098
method:rectangle,       integral:1.0979459183, error:0.00066637037050254477, error_order:1.18, error_change_rate:0.9997
method:trapezoid,       integral:1.0986125850, error:0.00000029629616404847, error_order:2.42, error_change_rate:2.0000
method:euler-maclaurin, integral:1.0986122887, error:0.00000000000013233858, error_order:4.77, error_change_rate:3.9338


$\int_1^{100} dx \frac{1}{x} = \log 100$

In [11]:
print_result(lambda x:1/x, lambda x:-1/(x*x), a=1, b=100, true_value=math.log(100), N=1000)  # note N=1000

true_value:4.605170185988092
method:rectangle,       integral:4.5569810575, error:0.04818912847341216832, error_order:1.31, error_change_rate:0.9878
method:trapezoid,       integral:4.6059860575, error:0.00081587152658801898, error_order:3.07, error_change_rate:1.9989
method:euler-maclaurin, integral:4.6051693892, error:0.00000079679841213931, error_order:6.07, error_change_rate:3.9950


In [12]:
print_result(lambda x:1/x, lambda x:-1/(x*x), a=1, b=100, true_value=math.log(100), N=10000)  # changed N to 1e4

true_value:4.605170185988092
method:rectangle,       integral:4.6002778526, error:0.00489233339678563084, error_order:1.15, error_change_rate:0.9988
method:trapezoid,       integral:4.6051783526, error:0.00000816660321412144, error_order:2.54, error_change_rate:2.0000
method:euler-maclaurin, integral:4.6051701859, error:0.00000000008003553376, error_order:5.04, error_change_rate:3.9931


$\int_0^3 dx e^x = e^3-1$

In [13]:
print_result(lambda x:math.exp(x), lambda x:math.exp(x), a=0, b=3, true_value=math.exp(3)-1, N=100)

true_value:19.085536923187668
method:rectangle,       integral:19.3732513708, error:0.28771444764628384405, error_order:0.36, error_change_rate:1.0036
method:trapezoid,       integral:19.0869683170, error:0.00143139379846957127, error_order:1.87, error_change_rate:2.0000
method:euler-maclaurin, integral:19.0855369017, error:0.00000002147076827441, error_order:5.04, error_change_rate:4.0000


In [14]:
print_result(lambda x:math.exp(x), lambda x:math.exp(x), a=0, b=3, true_value=math.exp(3)-1, N=1000)  # changed N to 1000

true_value:19.085536923187668
method:rectangle,       integral:19.1141795427, error:0.02864261953530800042, error_order:0.61, error_change_rate:1.0004
method:trapezoid,       integral:19.0855512373, error:0.00001431415052621787, error_order:1.92, error_change_rate:2.0000
method:euler-maclaurin, integral:19.0855369232, error:0.00000000000216715534, error_order:4.62, error_change_rate:4.1234


__Oh...__  
Althoug our test examples are rather simple and limited in number, 
 - in every case the error decreases in the order of rectangle, trapezoid, euler-maclaurin.
 - The error of rectangle method is of the order 0.5-2, trapezoid 1.5-2.5, euler-maclaurin 4.5-5.5, which roughly matches the theoretical expectation.
 - Moreover, when we decrease the step size $h$ to $h/2$, the error reduces by about $2^0, 2^{-2}, 2^{-4}$ for rectangle, trapezoid, euler-maclaurin, respectively.
   
__Oh, my oh...__